In [1]:
%%javascript
IPython.notebook.kernel.execute('notebook = "' + IPython.notebook.notebook_name + '"')

<IPython.core.display.Javascript object>

In [2]:
import os
with open(f'session.secret','r') as f:
    SESSION = f"session={f.read()}"
DAY = notebook.split('.')[0]
INPUT_FILE = f"{DAY}.txt"

In [3]:
# Manual INPUT
INPUT ="""INPUT"""

In [4]:
# Automatic INPUT
import os.path
if not os.path.isfile(INPUT_FILE):
    !curl https://adventofcode.com/2024/day/{DAY}/input --cookie {SESSION} > {INPUT_FILE}
with open(INPUT_FILE,'r') as f:
    INPUT = f.read().strip()

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100 22571  100 22571    0     0  56997      0 --:--:-- --:--:-- --:--:-- 56853


## Usefull import

In [5]:
import numpy as np
import pandas as pd
import networkx as nx
import copy as cp
import re

## Part 1

In [28]:
wall = set()
box = set()
robot = (-1,-1)
for x,row in enumerate(INPUT.split('\n\n')[0].strip().split('\n')):
    for y,c in enumerate(row):
        if c=='#':
            wall.add((x,y))
        if c=='@':
            robot=x,y
        if c=='O':
            box.add((x,y))

In [29]:
moves = INPUT.split('\n\n')[1].strip()
dxy = {'<':(0,-1),'>':(0,1),'^':(-1,0),'v':(1,0)}

In [30]:
def moveto(dx,dy,x,y):
    nx,ny = x+dx, y+dy
    if (x,y) in wall:
        return False
    if (x,y) in box:
        canmove= moveto(dx,dy,nx,ny)
        
        if canmove:
            box.remove((x,y))
            box.add((nx,ny))
        
        return canmove

    return True
            

In [31]:
for move in moves:
    if move not in dxy:
        continue
    dx,dy = dxy[move]
    nx,ny = robot[0]+dx, robot[1]+dy
    if moveto(dx,dy,nx,ny):
        robot = nx,ny
sum([100*bx+by for bx,by in box])

1499739

## Part 2
    

In [41]:
new_input=INPUT.replace('#','##').replace('.','..').replace('O','O.').replace('@','@.')
wall = set()
box = set()
robot = (-1,-1)
for x,row in enumerate(new_input.split('\n\n')[0].strip().split('\n')):
    for y,c in enumerate(row):
        if c=='#':
            wall.add((x,y))
        if c=='@':
            robot=x,y
        if c=='O':
            box.add((x,y))

In [42]:
def can_move(dx,dy,x,y):
    
    if (x,y) in wall:
        return False,set()
    if dy==-1 and (x,y-1) in box:
        cm,mb = can_move(dx,dy,x,y-2)
        return cm,mb | {(x,y-1)}
    elif dy==1 and (x,y) in box:
        cm,mb = can_move(dx,dy,x,y+2)
        return cm,mb | {(x,y)}
    elif dy==0 and (x,y) in box :
        cm,mb =  can_move(dx,dy,x+dx,y) 
        cm2,mb2 = can_move(dx,dy,x+dx,y+1)
        return cm and cm2, mb | mb2 | {(x,y)}
        
    elif dy==0 and (x,y-1) in box :
        cm,mb = can_move(dx,dy,x+dx,y) 
        cm2,mb2 =can_move(dx,dy,x+dx,y-1)
        return cm and cm2, mb | mb2 | {(x,y-1)}

    return True,set()

In [43]:
for move in moves:
    if move not in dxy:
        continue
    dx,dy = dxy[move]
    nx,ny = robot[0]+dx, robot[1]+dy
    cm,moving_boxes =  can_move(dx,dy,nx,ny)
    if cm:
        robot = nx,ny
        for b in moving_boxes:
            box.remove(b)
        for bx,by in moving_boxes:
            box.add((bx+dx,by+dy))
            
sum([100*bx+by for bx,by in box])

1522215